# Convolutional Neural Network

### Part 1 - Building the CNN

#### Importing the Tensorflow libraries and packages

In [4]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.6.0
Keras Version: 2.6.0

Python 3.8.0 (default, Nov  6 2019, 16:00:02) [MSC v.1916 64 bit (AMD64)]
Pandas 1.2.5
Scikit-Learn 1.0.2
GPU is available


In [5]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau

import os

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
classifier = Sequential()

#classes = 2

#Kernels usually 1x1 | 3x3 | 5x5

#Adding a convolutional layer
classifier.add(Conv2D(32, (3, 3), input_shape = (150, 150, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Adding a second convolutional layer
classifier.add(Conv2D(64, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.20))

#Adding a third convolutional layer
classifier.add(Conv2D(128, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.20))

#Adding a fourth convolutional layer
classifier.add(Conv2D(128, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Flattening
classifier.add(Flatten())


#Full connection
classifier.add(Dense(units = 512, activation = 'relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units = 1, activation= 'sigmoid'))

Let's double check the model description:

In [4]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 128)       0

In [5]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Part 2 - Fitting the CNN to the images


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

path_dir = '../dataset_mel_final/train'
parth_dir_test = '../dataset_mel_final/test'

datagen = ImageDataGenerator(
                    rescale=1./255,
                    validation_split = 0.2)

train_generator = datagen.flow_from_directory(
    path_dir,
    target_size=(150,150),
    shuffle=True,
    subset='training',
    batch_size = 64,
    class_mode = 'binary'
)
validation_generator = datagen.flow_from_directory(
    path_dir,
    target_size=(150,150),
    subset='validation',
    batch_size = 64,
    class_mode = 'binary'
)

test_set = datagen.flow_from_directory(parth_dir_test,
                                        target_size=(150,150),
                                        batch_size =64,
                                        class_mode = 'binary')


Found 1102 images belonging to 2 classes.
Found 274 images belonging to 2 classes.
Found 344 images belonging to 2 classes.


In [7]:
train_generator.class_indices

{'nonviolence': 0, 'violence': 1}

In [8]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import datetime

#rm -rf /logs/

es = EarlyStopping(monitor='val_loss', patience=50)
mc = ModelCheckpoint('best_model1.h5', monitor='val_loss', mode='min', save_best_only=True)
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

# Create a TensorBoard callback
log_dir = "logs\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [9]:
classifier.fit(train_generator,
                         epochs = 600,
                         callbacks = [tensorboard_callback,es,mc,reduceLR],
                         validation_data = validation_generator
                         )

Epoch 1/600
18/18 [==============================] - 12s 319ms/step - loss: 0.7378 - accuracy: 0.4909 - val_loss: 0.6923 - val_accuracy: 0.5474
Epoch 2/600
18/18 [==============================] - 4s 208ms/step - loss: 0.6863 - accuracy: 0.5563 - val_loss: 0.6623 - val_accuracy: 0.5876
Epoch 3/600
18/18 [==============================] - 4s 209ms/step - loss: 0.5686 - accuracy: 0.7250 - val_loss: 0.4999 - val_accuracy: 0.7664
Epoch 4/600
18/18 [==============================] - 4s 209ms/step - loss: 0.4585 - accuracy: 0.7958 - val_loss: 0.4600 - val_accuracy: 0.7555
Epoch 5/600
18/18 [==============================] - 4s 216ms/step - loss: 0.3791 - accuracy: 0.8385 - val_loss: 0.2156 - val_accuracy: 0.9270
Epoch 6/600
18/18 [==============================] - 4s 212ms/step - loss: 0.3195 - accuracy: 0.8811 - val_loss: 0.2045 - val_accuracy: 0.9416
Epoch 7/600
18/18 [==============================] - 4s 208ms/step - loss: 0.3106 - accuracy: 0.8721 - val_loss: 0.1738 - val_accuracy: 0.952

Epoch 58/600
18/18 [==============================] - 4s 215ms/step - loss: 0.0146 - accuracy: 0.9973 - val_loss: 0.1508 - val_accuracy: 0.9380
Epoch 59/600
18/18 [==============================] - 4s 214ms/step - loss: 0.0175 - accuracy: 0.9964 - val_loss: 0.1509 - val_accuracy: 0.9380
Epoch 60/600
18/18 [==============================] - 4s 214ms/step - loss: 0.0151 - accuracy: 0.9982 - val_loss: 0.1509 - val_accuracy: 0.9380
Epoch 61/600
18/18 [==============================] - 4s 217ms/step - loss: 0.0199 - accuracy: 0.9936 - val_loss: 0.1509 - val_accuracy: 0.9380
Epoch 62/600
18/18 [==============================] - 4s 218ms/step - loss: 0.0128 - accuracy: 0.9982 - val_loss: 0.1508 - val_accuracy: 0.9380
Epoch 63/600
18/18 [==============================] - 4s 218ms/step - loss: 0.0133 - accuracy: 0.9991 - val_loss: 0.1509 - val_accuracy: 0.9380
Epoch 64/600
18/18 [==============================] - 4s 217ms/step - loss: 0.0144 - accuracy: 0.9964 - val_loss: 0.1510 - val_accuracy:

In [10]:
#In terms of ‘loss’, overfitting reveals itself when your model has a low error
#in the training set and a higher error in the testing set.

#Overfitting indicates that your model is too complex for the problem that it is solving,
#filters in the case of Convolutional Neural Networks, and layers in the case of overall Deep Learning Models

In [6]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 11704), started 1 day, 21:53:12 ago. (Use '!kill 11704' to kill it.)

# Part 3 test the CNN

In [15]:
# Load Model
classifier.load_weights('best_model1.h5')

In [16]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 128)       0

In [17]:
score = classifier.evaluate(test_set, verbose=0)

print('Test Loss', score[0])
print('Test accuracy', score[1])

Test Loss 0.21472389996051788
Test accuracy 0.9244186282157898
